# Basic Classifying VAE for MNIST Database

In [1]:
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import torch
import pprint
import numpy as np
import numpy.random as random
import datetime
import matplotlib.pyplot as plt
from collections import defaultdict

import os
import sys
print("Current working dir: ", os.getcwd())
print("Initial path: ", sys.path)
sys.path.append("../")
print()
print("Updated path: ", sys.path)
from src.pytorch_cl_vae.model import ClVaeModel
print()
print("PyTorch Version ", torch.__version__)

Current working dir:  C:\Study\classifying_autoencoders\notebooks
Initial path:  ['', 'C:\\Study', 'C:\\Study\\openai\\baselines', 'C:\\ProgramData\\Anaconda3\\python36.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Babel-2.5.0-py3.6.egg', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\praat_formants_python-0.1.1-py3.6.egg', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\python_praat_scripts-0.2.1-py3.6.egg', 'c:\\study\\quadrotor_rl\\gym-aero', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\s3628075\\.ipython']

Updated path:  ['', 'C:\\Study', 'C:\\Study\\openai\\baselines', 'C:\\ProgramData\\Anaconda3\\python36

## 1 - Specify parameters for to the VAE and training

In [2]:
params = {
    'batch_size': 100,
    'num_epochs': 10,
    'latent_dim': 2,
    'encoder_hidden_size': 512,
    'decoder_hidden_size': 512,
    'classifier_hidden_size': 512,
    'vae_learning_rate': 0.0001,
    'classifier_learning_rate': 0.0001,
    'log_dir': '../data/logs',
    'model_dir': '../data/models',
    'data_dir': '../data'
}

## 2 - Fetch MNIST

In [3]:
mnist = fetch_mldata('MNIST original', data_home=params['data_dir'])
mnist.data = mnist.data / 255
num_samples, input_dim = mnist.data.shape
num_classes = len(np.unique(mnist.target))
lb = preprocessing.LabelBinarizer()
lb.fit(mnist.target)
params['classes_dim'] = [num_classes]
params['original_dim'] = input_dim
print('MNIST db has been successfully loaded, stored in the: "{}"'.format(params['data_dir'] + '/mldata'))
# split data to train and test subsets
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.1, random_state=0)
print("| Train subset shape:{} | Test subset shape:{} |".format(X_train.shape, X_test.shape))

MNIST db has been successfully loaded, stored in the: "../data/mldata"
| Train subset shape:(63000, 784) | Test subset shape:(7000, 784) |


## 3 - Create Model

In [4]:
# Initialize ClVaeModel
dev = "cuda:0" if torch.cuda.is_available() else "cpu"
print(dev)
params['device'] = dev
model = ClVaeModel(**params)
print("Model successfully initialized with params: ")
pprint.PrettyPrinter(indent=4).pprint(params)
print()
print()
print("Network Architecture:")
print()
print(model)

train_losses = []
train_accuracies = []

cuda:0
Model successfully initialized with params: 
{   'batch_size': 100,
    'classes_dim': [10],
    'classifier_hidden_size': 512,
    'classifier_learning_rate': 0.0001,
    'data_dir': '../data',
    'decoder_hidden_size': 512,
    'device': 'cuda:0',
    'encoder_hidden_size': 512,
    'latent_dim': 2,
    'log_dir': '../data/logs',
    'model_dir': '../data/models',
    'num_epochs': 10,
    'original_dim': 784,
    'vae_learning_rate': 0.0001}


Network Architecture:



## 4 - Train

In [6]:
save_each_steps = 500

# Train loop
train_step_i = 0
for epoch in range(params['num_epochs']):
    print('\nepoch {} out of {}'.format(epoch + 1, params['num_epochs']))
    for i in range(X_train.shape[0] // params['batch_size']):
        # Sample batch
        idx = random.choice(np.arange(0, X_train.shape[0]), params['batch_size'])
        x_batch = torch.from_numpy(X_train[idx]).float().to(torch.device(dev))
        y_batch = lb.transform(y_train[idx])
        y_batch = [torch.from_numpy(y_batch).float().to(torch.device(dev))]
        step_losses, step_accuracies = model.train_step(x_batch, y_batch)

#         step_losses = [loss.sum().detach().numpy() for loss in step_losses]
        # step_losses = Losses(*step_losses)
        # step_accuracies = Accuracies(*step_accuracies)

        train_losses.append(step_losses)
        train_accuracies.append(step_accuracies)

        train_step_i += 1

        print("\r|train step: {} | rec loss: {:.4f} | z_dkl loss: {:.4f} | class loss: {:.4f}"
              " | w_dkl loss: {:.4f} | class_accuracy: {:.4f} |".format(
            train_step_i, *step_losses, *step_accuracies
            ), end='')
        if train_step_i % 100 == 0:
            print()
        if train_step_i % save_each_steps == 0:
            dt = str(datetime.datetime.now().strftime("%m_%d_%Y_%I_%M_%p"))
            fname = params['model_dir'] + '/cl_vae_mnist_{}.pt'.format(dt)
            model.save_ckpt(fname)
model.save_ckpt(params['model_dir'] + '/cl_vae_mnist_last.pt')
print('*****Finished with the final loss: ', step_losses)


epoch 1 out of 10
|train step: 100 | rec loss: 0.2066 | z_dkl loss: 0.0053 | class loss: 1.8918 | w_dkl loss: 0.1120 | class_accuracy: 0.8200 |
|train step: 200 | rec loss: 0.2024 | z_dkl loss: 0.0077 | class loss: 1.9057 | w_dkl loss: 0.1253 | class_accuracy: 0.7700 |
|train step: 300 | rec loss: 0.1952 | z_dkl loss: 0.0230 | class loss: 1.8461 | w_dkl loss: 0.1013 | class_accuracy: 0.8800 |
|train step: 400 | rec loss: 0.2039 | z_dkl loss: 0.0203 | class loss: 1.8729 | w_dkl loss: 0.1137 | class_accuracy: 0.8400 |
|train step: 500 | rec loss: 0.2025 | z_dkl loss: 0.0128 | class loss: 1.8834 | w_dkl loss: 0.1532 | class_accuracy: 0.8300 |
|train step: 600 | rec loss: 0.1916 | z_dkl loss: 0.0134 | class loss: 1.8644 | w_dkl loss: 0.0486 | class_accuracy: 0.8800 |
|train step: 630 | rec loss: 0.1927 | z_dkl loss: 0.0248 | class loss: 1.8806 | w_dkl loss: 0.1123 | class_accuracy: 0.8200 |
epoch 2 out of 10
|train step: 700 | rec loss: 0.1968 | z_dkl loss: 0.0077 | class loss: 1.8927 | w

|train step: 5600 | rec loss: 0.1908 | z_dkl loss: 0.0150 | class loss: 1.8133 | w_dkl loss: 0.0981 | class_accuracy: 0.8900 |
|train step: 5670 | rec loss: 0.1810 | z_dkl loss: 0.0057 | class loss: 1.7781 | w_dkl loss: 0.0545 | class_accuracy: 0.9000 |
epoch 10 out of 10
|train step: 5700 | rec loss: 0.1948 | z_dkl loss: 0.0083 | class loss: 1.8098 | w_dkl loss: 0.0909 | class_accuracy: 0.8400 |
|train step: 5800 | rec loss: 0.1947 | z_dkl loss: 0.0266 | class loss: 1.8689 | w_dkl loss: 0.0670 | class_accuracy: 0.7800 |
|train step: 5900 | rec loss: 0.1903 | z_dkl loss: 0.0035 | class loss: 1.8108 | w_dkl loss: 0.0951 | class_accuracy: 0.8600 |
|train step: 6000 | rec loss: 0.1978 | z_dkl loss: 0.0067 | class loss: 1.8756 | w_dkl loss: 0.1042 | class_accuracy: 0.7800 |
|train step: 6100 | rec loss: 0.1909 | z_dkl loss: 0.0186 | class loss: 1.8200 | w_dkl loss: 0.0990 | class_accuracy: 0.8800 |
|train step: 6200 | rec loss: 0.2054 | z_dkl loss: 0.0042 | class loss: 1.8199 | w_dkl loss:

## 5 - Show losses graph

In [ ]:
%matplotlib inline
losses = defaultdict(list)
losses_names = train_losses[0]._fields
print(losses_names)
step_loss = train_losses[0]
print(*step_loss)
for i, loss_name in enumerate(losses_names):
    losses[loss_name] = [l[i] for l in train_losses]
    plt.figure()
    plt.title(loss_name)
    plt.plot(losses[loss_name])
    plt.legend()
plt.show()